# SageMaker LLM-as-Judge Evaluation - Basic Usage

This notebook demonstrates the basic user-facing flow for creating and managing LLM-as-Judge evaluation jobs using the LLMAsJudgeEvaluator.

## Configuration

In [1]:
# Configuration
REGION = 'us-west-2'
S3_BUCKET = 's3://mufi-test-serverless-smtj/eval/'
# DATASET = 'arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/gen-qa-test-content/1.0.1'  # Dataset ARN or S3 URI
DATASET = "s3://my-sagemaker-sherpa-dataset/dataset/gen-qa-formatted-dataset/gen_qa.jsonl"
MLFLOW_ARN = 'arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment'

## Step 1: Import Required Libraries

Import the LLMAsJudgeEvaluator class.

In [2]:
import json
from sagemaker.train.evaluate import LLMAsJudgeEvaluator
from rich.pretty import pprint

# Configure logging to show INFO messages
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(name)s - %(message)s'
)

## Step 2: Create LLMAsJudgeEvaluator

Create an LLMAsJudgeEvaluator instance with the desired evaluator model, dataset, and metrics.

### Key Parameters:
- `model`: Model Package (or Base Model) to be evaluated (required)
- `evaluator_model`: Bedrock model ID to use as judge (required)
- `dataset`: S3 URI or Dataset ARN (required)
- `builtin_metrics`: List of built-in metrics (optional, no 'Builtin.' prefix needed)
- `custom_metrics`: JSON string of custom metrics (optional)
- `evaluate_base_model`: Whether to evaluate base model in addition to custom model (optional, default=True)
- `mlflow_resource_arn`: MLflow tracking server ARN (optional)
- `model_package_group`: Model package group ARN (optional)
- `s3_output_path`: S3 output location (required)

### A. Using custom metrics (as JSON string)

Custom metrics must be provided as a properly escaped JSON string. You can either:
1. Create a Python dict and use `json.dumps()` to convert it
2. Provide a pre-escaped JSON string directly

In [3]:
# Method 1: Create dict and convert to JSON string
custom_metric_dict = {
    "customMetricDefinition": {
        "name": "PositiveSentiment",
        "instructions": (
            "You are an expert evaluator. Your task is to assess if the sentiment of the response is positive. "
            "Rate the response based on whether it conveys positive sentiment, helpfulness, and constructive tone.\n\n"
            "Consider the following:\n"
            "- Does the response have a positive, encouraging tone?\n"
            "- Is the response helpful and constructive?\n"
            "- Does it avoid negative language or criticism?\n\n"
            "Rate on this scale:\n"
            "- Good: Response has positive sentiment\n"
            "- Poor: Response lacks positive sentiment\n\n"
            "Here is the actual task:\n"
            "Prompt: {{prompt}}\n"
            "Response: {{prediction}}"
        ),
        "ratingScale": [
            {"definition": "Good", "value": {"floatValue": 1}},
            {"definition": "Poor", "value": {"floatValue": 0}}
        ]
    }
}

# Convert to JSON string
custom_metrics_json = json.dumps([custom_metric_dict])  # Note: wrap in list

In [ ]:

# Create evaluator with custom metrics
evaluator = LLMAsJudgeEvaluator(
    # base_model='arn:aws:sagemaker:us-west-2:052150106756:model-package/Demo-test-deb-2/1',  # Required
    model="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28",
    evaluator_model="anthropic.claude-3-5-haiku-20241022-v1:0",  # Required
    dataset=DATASET,  # Required: S3 URI or Dataset ARN
    builtin_metrics=["Completeness", "Faithfulness"],  # Optional: Can combine with custom metrics
    custom_metrics=custom_metrics_json,  # Optional: JSON string of custom metrics
    mlflow_resource_arn=MLFLOW_ARN,  # Optional
    # model_package_group=MODEL_PACKAGE_GROUP,  # Optional if BASE_MODEL is a Model Package ARN/Object
    s3_output_path=S3_BUCKET,  # Required
    evaluate_base_model=False
)

pprint(evaluator)

[11/29/25 13:43:52] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=406523;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=534480;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/mufi/Library/Application Support/sagemaker/config.yaml


                    INFO     Resolved MLflow resource ARN:                                    ]8;id=360312;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=805617;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#113\113]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

LLMAsJudgeEvaluator(
│   region=None,
│   sagemaker_session=<sagemaker.core.helper.session_helper.Session object at 0x15f5c11c0>,
│   model='arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28',
│   base_eval_name='eval-meta-04295d90',
│   s3_output_path='s3://mufi-test-serverless-smtj/eval/',
│   mlflow_resource_arn='arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment',
│   mlflow_experiment_name=None,
│   mlflow_run_name=None,
│   networking=None,
│   kms_key_id=None,
│   model_package_group=None,
│   evaluator_model='anthropic.claude-3-5-haiku-20241022-v1:0',
│   dataset='s3://my-sagemaker-sherpa-dataset/dataset/gen-qa-formatted-dataset/gen_qa.jsonl',
│   builtin_metrics=['Completeness', 'Faithfulness'],
│   custom_metrics='[{"customMetricDefinition": {"name": "PositiveSentiment", "instructions": "You are an expert evaluator. Your task is to assess if the sentiment of the response is positive. Rate the response based on whether it conveys positive sentiment, helpfulness, and constructive tone.\\n\\nConsider the following:\\n- Does the response have a positive, encouraging tone?\\n- Is the response helpful and constructive?\\n- Does it avoid negative language or criticism?\\n\\nRate on this scale:\\n- Good: Response has positive sentiment\\n- Poor: Response lacks positive sentiment\\n\\nHere is the actual task:\\nPrompt: {{prompt}}\\nResponse: {{prediction}}", "ratingScale": [{"definition": "Good", "value": {"floatValue": 1}}, {"definition": "Poor", "value": {"floatValue": 0}}]}}]',
│   evaluate_base_model=False
)

### [Optional] Example with multiple custom metrics

In [13]:
# # Create multiple custom metrics
# custom_metrics_list = [
#     {
#         "customMetricDefinition": {
#             "name": "GoodMetric",
#             "instructions": (
#                 "Assess if the response has positive sentiment. "
#                 "Prompt: {{prompt}}\nResponse: {{prediction}}"
#             ),
#             "ratingScale": [
#                 {"definition": "Good", "value": {"floatValue": 1}},
#                 {"definition": "Poor", "value": {"floatValue": 0}}
#             ]
#         }
#     },
#     {
#         "customMetricDefinition": {
#             "name": "BadMetric",
#             "instructions": (
#                 "Assess if the response has negative sentiment. "
#                 "Prompt: {{prompt}}\nResponse: {{prediction}}"
#             ),
#             "ratingScale": [
#                 {"definition": "Bad", "value": {"floatValue": 1}},
#                 {"definition": "Good", "value": {"floatValue": 0}}
#             ]
#         }
#     }
# ]

# # Convert list to JSON string
# custom_metrics_json = json.dumps(custom_metrics_list)

# # Create evaluator
# evaluator = LLMAsJudgeEvaluator(
#     base_model=BASE_MODEL,
#     evaluator_model="anthropic.claude-3-5-haiku-20241022-v1:0",
#     dataset=DATASET,
#     custom_metrics=custom_metrics_json,  # Multiple custom metrics
#     s3_output_path=S3_BUCKET,
# )

# print(f"✅ Created evaluator with {len(json.loads(custom_metrics_json))} custom metrics")
# pprint(evaluator)

### [Optional] Skipping base model evaluation (evaluate custom model only)

By default, LLM-as-Judge evaluates both the base model and custom model. You can skip base model evaluation to save time and cost by setting `evaluate_base_model=False`.

In [14]:
# # Define custom metrics (same as test script)
# custom_metrics = "[{\"customMetricDefinition\":{\"name\":\"GoodMetric\",\"instructions\":\"You are an expert evaluator. Your task is to assess if the sentiment of the response is positive. Rate the response based on whether it conveys positive sentiment, helpfulness, and constructive tone.\\n\\nConsider the following:\\n- Does the response have a positive, encouraging tone?\\n- Is the response helpful and constructive?\\n- Does it avoid negative language or criticism?\\n\\nRate on this scale:\\n- Good: Response has positive sentiment\\n- Poor: Response lacks positive sentiment\\n\\nHere is the actual task:\\nPrompt: {{prompt}}\\nResponse: {{prediction}}\",\"ratingScale\":[{\"definition\":\"Good\",\"value\":{\"floatValue\":1}},{\"definition\":\"Poor\",\"value\":{\"floatValue\":0}}]}},{\"customMetricDefinition\":{\"name\":\"BadMetric\",\"instructions\":\"You are an expert evaluator. Your task is to assess if the sentiment of the response is negative. Rate the response based on whether it conveys negative sentiment, unhelpfulness, or destructive tone.\\n\\nConsider the following:\\n- Does the response have a negative, discouraging tone?\\n- Is the response unhelpful or destructive?\\n- Does it use negative language or harsh criticism?\\n\\nRate on this scale:\\n- Bad: Response has negative sentiment\\n- Good: Response lacks negative sentiment\\n\\nHere is the actual task:\\nPrompt: {{prompt}}\\nResponse: {{prediction}}\",\"ratingScale\":[{\"definition\":\"Bad\",\"value\":{\"floatValue\":1}},{\"definition\":\"Good\",\"value\":{\"floatValue\":0}}]}}]"

# # Create evaluator that only evaluates the custom model (matching test script exactly)
# evaluator = LLMAsJudgeEvaluator(
#     base_model=BASE_MODEL,
#     evaluator_model="anthropic.claude-3-5-haiku-20241022-v1:0",
#     dataset=DATASET,
#     builtin_metrics=["Completeness", "Faithfulness", "Helpfulness"],
#     custom_metrics=custom_metrics,
#     mlflow_resource_arn=MLFLOW_ARN,
#     model_package_group=MODEL_PACKAGE_GROUP,
#     model_artifact=MODEL_ARTIFACT,
#     s3_output_path=S3_BUCKET,
#     evaluate_base_model=False,  # KEY: Skip base model evaluation
# )

# print("✅ Created evaluator (custom model only)")
# pprint(evaluator)

## Step 3: Run LLM-as-Judge Evaluation

Start the evaluation job. The evaluator will:
1. Generate inference responses from the base model (if evaluate_base_model=True)
2. Generate inference responses from the custom model
3. Use the judge model to evaluate responses with built-in and custom metrics

In [8]:
# Run evaluation
execution = evaluator.evaluate()

print(f"✅ Evaluation job started!")
print(f"Job ARN: {execution.arn}")
print(f"Job Name: {execution.name}")
print(f"Status: {execution.status.overall_status}")

pprint(execution)

[11/29/25 16:22:01] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=931878;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=760856;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Getting or creating artifact for source:                         ]8;id=179503;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=71430;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#597\597]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package/test-fine                      
                             tuned-models-gamma/28                                                                 

                    INFO     Searching for existing artifact for model package:               ]8;id=2444;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=787547;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#459\459]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package/test-fine                      
                             tuned-models-gamma/28                                                                 

                    INFO     Found existing artifact:                                         ]8;id=808361;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=665812;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#468\468]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:artifact/2b64ef9fe915b3                      
                             138877d772ec489bef                                                                    

                    INFO     Inferred model package group ARN:                                ]8;id=361400;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=518747;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#386\386]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package-group/tes                      
                             t-finetuned-models-gamma from                                                         
                             arn:aws:sagemaker:us-west-2:052150106756:model-package/test-fine                      
                             tuned-models-gamma/28                                                                 

...

[11/29/25 16:22:03] INFO     Successfully updated pipeline:                                        ]8;id=916795;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=385336;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#208\208]8;;\
                             SagemakerEvaluation-LLMAJEvaluation-f952b79f-4afe-4f2f-b45d-17894533c                 
                             6e9                                                                                   

                    INFO     Starting pipeline execution: eval-meta-04295d90-1764462123            ]8;id=41189;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=464412;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#263\263]8;;\

                    INFO     Pipeline execution started:                                           ]8;id=227887;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=844359;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#274\274]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation                 
                             -LLMAJEvaluation-f952b79f-4afe-4f2f-b45d-17894533c6e9/execution/m318n                 
                             ngjk32f                                                                               

✅ Evaluation job started!
Job ARN: arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-LLMAJEvaluation-f952b79f-4afe-4f2f-b45d-17894533c6e9/execution/m318nngjk32f
Job Name: eval-meta-04295d90
Status: Executing


LLMAJEvaluationExecution(
│   arn='arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-LLMAJEvaluation-f952b79f-4afe-4f2f-b45d-17894533c6e9/execution/m318nngjk32f',
│   name='eval-meta-04295d90',
│   status=PipelineExecutionStatus(overall_status='Executing', step_details=[], failure_reason=None),
│   last_modified_time=datetime.datetime(2025, 11, 29, 16, 22, 3, 689000, tzinfo=tzlocal()),
│   eval_type=<EvalType.LLM_AS_JUDGE: 'llmasjudge'>,
│   s3_output_path='s3://mufi-test-serverless-smtj/eval/',
│   steps=[]
)

## Step 4: Check Job Status

Refresh and display the current job status with step details.

In [9]:
# Refresh status
execution.refresh()

# Display job status using rich pprint
pprint(execution.status)

PipelineExecutionStatus(
│   overall_status='Executing',
│   step_details=[
│   │   StepDetail(
│   │   │   name='CreateEvaluationAction',
│   │   │   status='Starting',
│   │   │   start_time='2025-11-29T16:22:04.148000-08:00',
│   │   │   end_time='<sagemaker.core.utils.utils.Unassigned object at 0x1298e7170>',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   )
│   ],
│   failure_reason=None
)

## Step 5: Monitor Pipeline Execution

Poll the pipeline status until it reaches a terminal state (Succeeded, Failed, or Stopped).

In [10]:
# Wait for job completion (optional)
# This will poll every 5 seconds for up to 1 hour
execution.wait(poll=5, timeout=3600)

╭─────────────────────────────────────────── Pipeline Execution Status ───────────────────────────────────────────╮
│  Overall Status        Succeeded                                                                                │
│  Target Status         Succeeded                                                                                │
│  Elapsed Time          1885.8s                                                                                  │
│                                                                                                                 │
│ Pipeline Steps                                                                                                  │
│  Step Name                       Status           Duration                                                      │
│  AssociateLineage                Succeeded        1.9s                                                          │
│  EvaluateCustomModelMetrics      Succeeded        1327.1s                                                       │
│  EvaluateCustomInferenceModel    Succeeded        554.1s                                                        │
│  CreateEvaluationAction          Succeeded        4.5s                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[11/29/25 16:53:37] INFO     Final Resource Status: Succeeded                                      ]8;id=524139;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=278480;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#979\979]8;;\

In [11]:
# Display results
execution.show_results(limit=10, offset=0, show_explanations=False)

[11/29/25 17:02:07] INFO     Extracted training job name:                                  ]8;id=177834;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=168478;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#52\52]8;;\
                             pipelines-m318nngjk32f-EvaluateCustomModelM-lN73ONZ955 from                           
                             step: EvaluateCustomModelMetrics (priority: Custom)                                   

╭─────────────────────────────────────────── Result Artifacts Location ───────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  📦 Full evaluation artifacts available at:                                                                     │
│    s3://mufi-test-serverless-smtj/eval/pipelines-m318nngjk32f-EvaluateCustomModelM-lN73ONZ955/                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                    INFO     S3 bucket: mufi-test-serverless-smtj, prefix: eval           ]8;id=453165;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=425984;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#341\341]8;;\

                    INFO     Extracted training job name:                                  ]8;id=324161;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=683512;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#52\52]8;;\
                             pipelines-m318nngjk32f-EvaluateCustomModelM-lN73ONZ955 from                           
                             step: EvaluateCustomModelMetrics (priority: Custom)                                   

                    INFO     Searching for bedrock summary in                             ]8;id=308182;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=660550;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#361\361]8;;\
                             s3://mufi-test-serverless-smtj/eval/pipelines-m318nngjk32f-E                          
                             valuateCustomModelM-lN73ONZ955/output/output/                                         

...

Model Response: I ate a hamburger today and it was kind of dry.

 Metric                                Score  
 ───────────────────────────────────────────── 
  Builtin.Completeness                   0.0%  
  Builtin.Faithfulness                   0.0% 

══════════════════════════════════════════════════════════════════════

Showing evaluations 1-3 of 3

══════════════════════════════════════════════════════════════════════

## Retrieve an Existing Job

You can retrieve and inspect any existing evaluation job using its ARN.

In [ ]:
# Get an existing job by ARN
# Replace with your actual pipeline execution ARN
existing_arn = 'arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-llmasjudge/execution/4hr7446yft1d'  # or use a specific ARN

from sagemaker.train.evaluate import EvaluationPipelineExecution
from rich.pretty import pprint

existing_execution = EvaluationPipelineExecution.get(
    arn=existing_arn,
    region="us-west-2"
)
pprint(existing_execution.status)

existing_execution.show_results(limit=5, offset=0, show_explanations=False)

[11/29/25 17:02:15] WARNING  Could not extract eval_type from ARN:                                 ]8;id=315627;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=953607;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#146\146]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation                 
                             -llmasjudge/execution/4hr7446yft1d                                                    

                    INFO     Extracted s3_output_path from training job                            ]8;id=739992;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=203397;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#367\367]8;;\
                             pipelines-4hr7446yft1d-EvaluateCustomModelM-qePWbkcMxz:                               
                             s3://mufi-test-serverless-smtj/eval                                                   

                    WARNING  Could not extract eval_type from ARN:                                 ]8;id=550335;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=858100;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#146\146]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation                 
                             -llmasjudge                                                                           

                    WARNING  Could not extract eval_type from ARN:                                 ]8;id=379628;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=725705;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#146\146]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation                 
                             -llmasjudge/execution/4hr7446yft1d                                                    

PipelineExecutionStatus(
│   overall_status='Succeeded',
│   step_details=[
│   │   StepDetail(
│   │   │   name='AssociateLineage',
│   │   │   status='Succeeded',
│   │   │   start_time='2025-11-19T15:45:57.889000-08:00',
│   │   │   end_time='2025-11-19T15:45:59.266000-08:00',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   ),
│   │   StepDetail(
│   │   │   name='EvaluateCustomModelMetrics',
│   │   │   status='Succeeded',
│   │   │   start_time='2025-11-19T15:27:55.641000-08:00',
│   │   │   end_time='2025-11-19T15:45:56.749000-08:00',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   ),
│   │   StepDetail(
│   │   │   name='EvaluateCustomInferenceModel',
│   │   │   status='Succeeded',
│   │   │   start_time='2025-11-19T15:18:07.804000-08:00',
│   │   │   end_time='2025-11-19T15:27:54.474000-08:00',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   ),
│   │   StepDetail(
│   │   │   name='CreateEvaluationAction',
│   │   │   status='Succeeded',
│   │   │   start_time='2025-11-19T15:18:05.550000-08:00',
│   │   │   end_time='2025-11-19T15:18:07.332000-08:00',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   )
│   ],
│   failure_reason=None
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 )                                                                                           │
│   15 pprint(existing_execution.status)                                                           │
│   16                                                                                             │
│ ❱ 17 existing_execution.show_results(limit=5, offset=0, show_explanations=False)                 │
│   18                                                                                             │
│                                                                                                  │
│ /Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/pydantic/main │
│ .py:1026 in __getattr__                                                                          │
│                                                                                                  │
│   1023 │   │   │   │   │   │   return super().__getattribute__(item)  # Raises AttributeError i  │
│   1024 │   │   │   │   │   else:                                                                 │
│   1025 │   │   │   │   │   │   # this is the current error                                       │
│ ❱ 1026 │   │   │   │   │   │   raise AttributeError(f'{type(self).__name__!r} object has no att  │
│   1027 │   │                                                                                     │
│   1028 │   │   def __setattr__(self, name: str, value: Any) -> None:                             │
│   1029 │   │   │   if (setattr_handler := self.__pydantic_setattr_handlers__.get(name)) is not   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'EvaluationPipelineExecution' object has no attribute 'show_results'

## Get All LLM-as-Judge Evaluations

Retrieve all LLM-as-Judge evaluation jobs.

In [13]:
from sagemaker.train.evaluate import LLMAsJudgeEvaluator

# Get all LLM-as-Judge evaluations as an iterator
all_executions = list(LLMAsJudgeEvaluator.get_all(region="us-west-2"))

print(f"Found {len(all_executions)} LLM-as-Judge evaluation jobs")
for execution in all_executions:
    print(f"  - {execution.name}: {execution.status.overall_status}")

[11/29/25 17:02:21] INFO     Extracted s3_output_path from training job                            ]8;id=802368;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=75226;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#367\367]8;;\
                             pipelines-m318nngjk32f-EvaluateCustomModelM-lN73ONZ955:                               
                             s3://mufi-test-serverless-smtj/eval                                                   

Found 2 LLM-as-Judge evaluation jobs
  - m318nngjk32f: Succeeded
  - 2m5hczli7vdp: Failed


## Stop a Running Job (Optional)

If needed, you can stop a running evaluation job.

In [ ]:
# Uncomment to stop the job
# execution.stop()
# print(f"Execution stopped. Status: {execution.status.overall_status}")

## Dataset Support

The `dataset` parameter supports two formats:

### 1. S3 URI
```python
dataset="s3://my-bucket/path/to/dataset.jsonl"
```

### 2. Dataset ARN (AI Registry)
```python
dataset="arn:aws:sagemaker:us-west-2:123456789012:hub-content/AIRegistry/DataSet/my-dataset/1.0.0"
```

The evaluator automatically detects which format is provided and uses the appropriate data source configuration.